In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm

In [2]:
def resize_threshold(image,resize_size=64):
    
    image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    ret, image = cv2.threshold(image_bw, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(image,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[-2:] # finding the dark countour
    
    idx = 0 
    ls_xmin = []
    ls_ymin = []
    ls_xmax = []
    ls_ymax = []
    for cnt in contours:
        idx += 1
        x,y,w,h = cv2.boundingRect(cnt)
        ls_xmin.append(x)
        ls_ymin.append(y)
        ls_xmax.append(x + w)
        ls_ymax.append(y + h)
    xmin = min(ls_xmin)
    ymin = min(ls_ymin)
    xmax = max(ls_xmax)
    ymax = max(ls_ymax)
    
    roi = image[ymin:ymax,xmin:xmax]
    resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
    
    return resized_roi

In [3]:
def create_processed_image_directory(dirname):
    
    datadir_raw_images = Path('/home/sayan/Documents/Bengali_Grapheme/images/toy_raw_images/')
    for (directory, _ , image_names) in os.walk(datadir_raw_images):
        for image_name in tqdm(image_names):
            image = cv2.imread(os.path.join(directory, image_name))
            image_denoise = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
            image_th = resize_threshold(image_denoise)
            img_save_dir = dirname + '/' + image_name.replace('.png','') + "_f.png"
            cv2.imwrite(img_save_dir, image_th.astype(float))

In [4]:
create_processed_image_directory('/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images/')

100%|██████████| 20084/20084 [29:47<00:00, 11.23it/s]
